In [ ]:
# LIBRERIA PARA SEPARAR AUDIO
# https://pydub.com/

### Abrir archivos

In [ ]:
path = 'sonido-presion/'
files = {i: {'presion': [], 'sonido': []} for i in range(1,4)}

for i,file in enumerate(os.listdir(path)):
  file_name = str(os.path.splitext(file)[0])
  segmento = int(file_name.split('_')[2])
  param = file_name.split('_')[0]

  sample_rate, data = wavfile.read(os.path.join(path, file))
  print(f'La frecuencia de muestreo del segmento {segmento} de {file_name.split("_")[0]} es {sample_rate}. N = {len(data)}')
  files[segmento][param] = data.astype(np.int32)

### Inteprolar

In [ ]:
from scipy.interpolate import interp1d

p = files[1]['presion']
s = files[1]['sonido']

interp_func = interp1d(t_m, p)

b_interpolado = interp_func(np.linspace(0, t_m[-1],len(t) ))

### Cotnruye un $\delta$ con dos segmentos lo más separados posibles

In [ ]:
# ACA FUERZO A QUE LOS DOS SEGMENTOS ESTEN UNO AL PRINCIPIO Y OTRO AL FINAL

def construir_delta2(xs, umbral, min_largo=20, margen=0.25):
    """
    Lo que hace el algoritmo es:
    1. Buscar todos los segmentos que superan el umbral.

    2. Filtrar para que uno esté cerca del principio y otro cerca del final de xs.

    3. Elegir los más largos dentro de esas dos regiones.

    """
    preliminar = (xs > umbral).astype(int)
    cambios = np.diff(preliminar)
    inicios = np.where(cambios == 1)[0] + 1
    finales = np.where(cambios == -1)[0] + 1

    if preliminar[0] == 1:
        inicios = np.insert(inicios, 0, 0)
    if preliminar[-1] == 1:
        finales = np.append(finales, len(xs))

    segmentos = [(ini, fin, fin - ini) for ini, fin in zip(inicios, finales) if (fin - ini) >= min_largo]
    if len(segmentos) < 2:
        return np.zeros_like(xs, dtype=int)

    N = len(xs)
    umbral_inicio = int(N * margen)
    umbral_final = int(N * (1 - margen))

    # Segmento cerca del principio
    candidatos_inicio = [s for s in segmentos if s[0] <= umbral_inicio]
    # Segmento cerca del final
    candidatos_final = [s for s in segmentos if s[1] >= umbral_final]

    if not candidatos_inicio or not candidatos_final:
        return np.zeros_like(xs, dtype=int)

    # Elegimos los más largos de cada grupo
    seg_inicio = max(candidatos_inicio, key=lambda s: s[2])
    seg_final = max(candidatos_final, key=lambda s: s[2])

    delta2 = np.zeros_like(xs, dtype=int)
    delta2[seg_inicio[0]:seg_inicio[1]] = 1
    delta2[seg_final[0]:seg_final[1]] = 1

    return delta2

def encontrar_mejor_umbral(xs, delta1, umbrales, min_largo=20):
    mejor_umbral = None
    mejor_delta2 = None
    mejor_score = -np.inf

    for u in umbrales:
        delta2 = construir_delta2(xs, u, min_largo)

        # Métrica: favorecer solapamiento, penalizar falsos positivos
        solap = np.sum(delta1 * delta2)
        penal = np.sum(delta2 * (1 - delta1)) * .8
        score = solap - penal  # puedes ajustar pesos si querés

        if score > mejor_score:
            mejor_score = score
            mejor_delta2 = delta2
            mejor_umbral = u

    return mejor_umbral, mejor_delta2, mejor_score